#### **Fundamentos de Bancos de Dados Relacionais e NoSQL**

#### **Conteúdo - Bases e Notebook da aula**

https://github.com/FIAP/Pos_Tech_DTAT/tree/main/Fase%203

#### **Importação de pacotes, bibliotecas e funções (def)**

In [ ]:
# Importar biblioteca completa
import boto3
import pandas as pd
import os

# Importar função especifica de um módulo
from botocore.exceptions import BotoCoreError, ClientError
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

In [ ]:
# Testar a conexão ao banco de dados
def test_connection(engine):

    try:
        with engine.connect() as connection:
            
            # Testar a versão do PostgreSQL
            result = connection.execute(text("SELECT version();"))
            versao = result.fetchone()
            print("✅ Conectado com sucesso:", versao[0])

            # Listar as tabelas no schema público
            result = connection.execute(text("""
                SELECT table_name
                FROM information_schema.tables
                WHERE table_schema = 'public';
            """))
            tabelas = result.fetchall()
            print("📄 Tabelas no banco:")
            for tabela in tabelas:
                print("  -", tabela[0])

    except Exception as e:
        print("❌ Erro ao executar comandos:", e)


#### **Instalação do AWS CLI (Opcional, mas Recomendado)**

Para rodar comandos da AWS no terminal para testar credenciais, criar buckets ou consultar recursos instale o **AWS CLI v2**.

##### **macOS / Linux (zsh ou bash)**

**Instalação com Homebrew (recomendado para macOS)**
```bash
brew install awscli
```

##### **Instalação manual**
```bash
curl "https://awscli.amazonaws.com/AWSCLIV2.pkg" -o "AWSCLIV2.pkg"
sudo installer -pkg AWSCLIV2.pkg -target /
```

##### **Verificar instalação**
```bash
aws --version
```

##### **Windows**

1. Baixe o instalador: [AWS CLI v2](https://awscli.amazonaws.com/AWSCLIV2.msi)
2. Execute o instalador e finalize.
3. Teste no PowerShell:
```bash
aws --version
```

##### **Testar credenciais**
Após instalar o **AWS CLI**, teste o perfil `academy` criado:

```bash
aws sts get-caller-identity --profile academy
```

Se retornar o **Account ID** e o **ARN**, as credenciais estão funcionando ✅

#### **Credenciais do AWS Academy** 

Para seguir com as etapas desse notebook será necessario pegar as credencias do AWS Academy e criar o arquivo com essas credencias:

**1. Pegar credenciais do AWS Academy**  
1. Acesse [AWS Academy](https://awsacademy.instructure.com/)
2. Acesse a opção **Cursos** no lado esquerdo e acesse o seu respectivo curso
3. Acessa a opçãp **Módulos**
4. Acesse a opção **Iniciar os laboratórios de aprendizagem da AWS Academy**
5. Clique em **Start Lab**
6. Clique em **AWS Details**
7. Clique em **AWS CLI: Show** e copie:
   - **AWS Access Key ID**
   - **AWS Secret Access Key**
   - **Session Token**

⚠️ As credenciais são temporárias (válidas por 3 horas).

**2. Criar arquivo de credenciais**  

Crie o arquivo **`credentials`** no caminho:

- **macOS/Linux:** `~/.aws/credentials`
- **Windows:** `%USERPROFILE%\.aws\credentials`

⚠️ Salvar o arquivo sem formato --> Na opçãp Tipo deixar Todos os arquivos

**3. Conteúdo do arquivo**
```ini
[academy]
aws_access_key_id = SUA_ACCESS_KEY
aws_secret_access_key = SUA_SECRET_KEY
aws_session_token = SEU_SESSION_TOKEN
region = sa-east-1
output = json
```

**4. Testar configuração**
```bash
aws sts get-caller-identity --profile academy
```

Se tudo estiver certo, rode o notebook.  
Agora ele **usa automaticamente o perfil `academy`** para se conectar à AWS.

#### **Testar conexão AWS via Python**

In [ ]:
# Validar conexão
try:
    session = boto3.Session(profile_name="academy")
    sts = session.client("sts")
    identity = sts.get_caller_identity()
    print("✅ Conectado à conta\n")
    print("UserId:", identity["UserId"])
    print("Account:", identity["Account"])
    print("Arn:", identity["Arn"])

except (BotoCoreError, ClientError) as e:
    print("❌ Erro ao conectar à AWS. Verifique suas credenciais e tente novamente.")
    print("Detalhes do erro:", e)


#### **Configuração do PostgreSQL na AWS RDS**

##### 1. **Criar instância RDS com PostgreSQL (SandBox)**

1. Acesse o console AWS → [https://us-east-1.console.aws.amazon.com/rds/home?region=us-east-1#](https://console.aws.amazon.com/rds/)
2. Clique em **Criar banco de dados**
3. Selecione:
   - **Tipo de banco:** PostgreSQL
   - **Versão:** PostgreSQL 15 (ou mais recente)
   - **Modelo de uso:** SandBox
   - **Identificador da instância:** `postgres-db`
   - **Usuário:** `postgres`
   - **Senha:** crie uma senha segura
4. Tipo de instância: `db.t3.micro`
5. Armazenamento: 20 GB (SSD General Purpose)
6. **Acesso público:** Habilitado (Sim)
7. **Nome do banco de dados inicial:** `db_relacional`
8. Clique em **Criar banco de dados**

##### **2. Liberar o IP na VPC / Grupo de Segurança (Security Group)**

1. Vá para **EC2 > Grupos de Segurança**
2. Encontre o grupo associado à instância RDS
3. Clique em **Editar regras de entrada**
4. Adicione uma nova regra:
   - Tipo: `PostgreSQL`
   - Porta: `5432`
   - Origem: `Seu IP` (ou `0.0.0.0/0` temporariamente para teste – cuidado com isso em produção)
5. Salve as alterações.

✅ Agora o acesso externo ao banco estará liberado para seu IP

##### **3. Copie o Endpoint da RDS**

1. Volte ao RDS > Banco de dados > `bd-relacional`
2. Copie o valor do campo **Endpoint** (algo como `bd-relacional.xxxxxx.us-east-1.rds.amazonaws.com`)
3. Use esse endpoint no notebook para se conectar com o PostgreSQL

#### **Aula 3 - Stacks Modernas, Data Warehouse, Data Lake e Lakehouse, Data Mesh e Data Fabric**

##### **Modelo Dimensional em DBML – Diagrama DW (dimensões e fato)**

Criar o Diagrama Entidade-Relacionamento (DER) usando o site https://dbdiagram.io/home e o seguinte código DBML (Database Markup Language ou Linguagem de Marcação para Banco de Dados):   

```dbml
Table dim_cliente {
  id_cliente int [pk]
  nome varchar
  idade int
  cidade varchar
}

Table dim_produto {
  id_produto int [pk]
  nome_produto varchar
  categoria varchar
  preco decimal
}

Table fato_pedidos {
  id_pedido int [pk]
  id_cliente int [ref: > dim_cliente.id_cliente]
  id_produto int [ref: > dim_produto.id_produto]
  id_data int [ref: > dim_data.id_data]
  id_regiao int [ref: > dim_regiao.id_regiao]
  data_pedido date
  quantidade int
  valor_total decimal
}

Table dim_data {
  id_data int [pk]
  data date
  ano int
  mes int
  dia int
  dia_semana varchar
  nome_mes varchar
}

Table dim_regiao {
  id_regiao int [pk]
  nome_regiao varchar
  estado varchar
  cidade varchar
}


![Diagrama DW](imagame.png)


##### **Conectar ao PostgreSQL via RDS**

In [ ]:
# Verificar variaveis de ambiente
print("Usuario: ", os.getenv("POSTGRES_USER"))
print("host: ", os.getenv("POSTGRES_HOST"))
print("porta: ", os.getenv("POSTGRES_PORT"))
print("banco: ", os.getenv("POSTGRES_DB"))

In [ ]:
# Criar a engine para conexão ao banco de dados usando .env

load_dotenv()

usuario = os.getenv("POSTGRES_USER")
senha = os.getenv("POSTGRES_PASSWORD")
host = os.getenv("POSTGRES_HOST")
porta = os.getenv("POSTGRES_PORT")
banco = os.getenv("POSTGRES_DB")

engine = create_engine(f"postgresql+psycopg2://{usuario}:{senha}@{host}:{porta}/{banco}")

In [ ]:
# Testar a conexão
test_connection(engine)

In [ ]:
# Listar de comandos individuais
ddl_commands = [
    """
    CREATE TABLE IF NOT EXISTS tipos_produto (
      id_tipo SERIAL PRIMARY KEY,
      nome_tipo VARCHAR(50) NOT NULL
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS produtos (
      id_produto SERIAL PRIMARY KEY,
      nome_produto VARCHAR(100) NOT NULL,
      preco DECIMAL(10,2) NOT NULL,
      id_tipo INT REFERENCES tipos_produto(id_tipo)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS clientes (
      id_cliente SERIAL PRIMARY KEY,
      nome VARCHAR(100) NOT NULL,
      email VARCHAR(100),
      telefone VARCHAR(20), 
      cidade VARCHAR(100) NOT NULL, 
      estado VARCHAR(2) NOT NULL
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS pedidos (
      id_pedido SERIAL PRIMARY KEY,
      data_pedido DATE NOT NULL,
      status VARCHAR(20) NOT NULL,
      id_cliente INT NOT NULL REFERENCES clientes(id_cliente)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS itens_pedido (
    id_item SERIAL PRIMARY KEY,
    id_pedido INT NOT NULL,
    id_produto INT NOT NULL,
    quantidade INT NOT NULL,
    preco_unitario DECIMAL(10,2) NOT NULL,
    CONSTRAINT fk_pedido FOREIGN KEY (id_pedido) REFERENCES pedidos(id_pedido) ON DELETE CASCADE,
    CONSTRAINT fk_produto FOREIGN KEY (id_produto) REFERENCES produtos(id_produto) ON DELETE CASCADE
    );

    """
]

# Executar comandos um a um
with engine.begin() as conn:
    for cmd in ddl_commands:
        conn.execute(text(cmd))

print("✅ Tabelas criadas com sucesso!")

In [ ]:
# Listar as tabelas
with engine.connect() as conn:
    result = conn.execute(text("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';"))
    for row in result:
        print(row[0])